- Poti centra si alinia imaginile cu SIFT si apoi sa faci crop. Cred ca ar trebui sa se ajunga la aceleasi coordonate dupa SIFT
- Filtrezi liniile astfel incat sa fie la vreo 80 de pixeli distanta

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
def show_image(image_, window_name='image', timeout=0):
    """
    Show image.
    :param image_
    :param window_name
    :param timeout
    """
    # h, w = image_.shape[0:2]
    # neww = 1024
    # newh = int(neww * (h / w))
    # img = cv.resize(image_, (neww, newh))

    cv.imshow(window_name, image_)
    cv.waitKey(timeout)
    cv.destroyAllWindows()

In [3]:
def get_keypoints_and_features(image, show_details = False) -> tuple:
    """
    
    :param image.
    :return the keypoints: [cv.Keypoint] and the features: np.ndarray for each keypoint.
    """
    
    def show_keypoints(image_, keypoints_):
        """
        Show the keypoints found in the image.
        """
        image_output = image_.copy()
        image_output = cv.drawKeypoints(image, keypoints_, image_output, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        show_image(image_output, 'keypoints')
        # cv.imwrite('keypoints.jpg', image_output)
    
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    sift = cv.SIFT_create()
    
    keypoints, features = sift.detectAndCompute(gray_image, None)
     
    if show_details:
        show_keypoints(image, keypoints)
        
    return keypoints, features

In [4]:
image = cv.imread(r'board+dominoes\02.jpg')
k, f = get_keypoints_and_features(image, True)

In [5]:
def align_images(img1, img2, good_match_percent=0.75, ref_filename=''):
    k1, f1 = get_keypoints_and_features(img1)
    k2, f2 = get_keypoints_and_features(img2)

    matcher = cv.DescriptorMatcher_create('FlannBased')
    # matches = matcher.match(f1, f2, None)
    all_matches = matcher.knnMatch(f1, f2, k=2) 

    # Add the best matches in a list
    matches = [match[0] for match in all_matches if match[0].distance / match[1].distance < good_match_percent]

    # list(matches).sort(key=lambda x: x.distance, reverse=False)
 
    # num_good_matches = int(len(matches) * good_match_percent)
    # matches = matches[:num_good_matches]

    # Draw top matches
    im_matches = cv.drawMatches(img1, k1, img2, k2, matches, None)
    if ref_filename != '':
        cv.imwrite("matches_" + ref_filename + ".jpg", im_matches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = k1[match.queryIdx].pt
        points2[i, :] = k2[match.trainIdx].pt

    # Find homography
    h, _ = cv.findHomography(points1, points2, cv.RANSAC)

    # Use homography
    height, width, _ = img2.shape
    im1_reg = cv.warpPerspective(img1, h, (width, height))

    return im1_reg, h

In [6]:
def save_aligned_image(ref_filename, image_path, out_path='aligned_train\\'):
    im_reference = cv.imread(ref_filename, cv.IMREAD_COLOR)

    if not os.path.exists(out_path):
        os.mkdir(out_path) 

    # Read image to be aligned
    im = cv.imread(image_path, cv.IMREAD_COLOR)

    # Registered image will be stored in im_reg.
    # The estimated homography will be stored in h.
    im_reg, h = align_images(im, im_reference)

    # Crop image
    cropped_image = im_reg[273 * 2 : (273 + 960) * 2,
                           547 * 2 : (547 + 956) * 2]
    # Write aligned image to disk.
    out_filename = "aligned_" + image_path.split('\\')[-1]
    cv.imwrite(out_path + out_filename, cropped_image)

def save_aligned_images(ref_filename='board+dominoes\\02.jpg', images_path='train\\regular_tasks\\', out_path='aligned_train\\'):
    print("Reading reference image : " + ref_filename)
    # Read images to be aligned
    img_filenames = [images_path + f for f in os.listdir(images_path) if f.endswith('.jpg')]
    for idx, img_filename in enumerate(img_filenames):
        save_aligned_image(ref_filename, img_filename, out_path)
        progress_bar(idx, len(img_filenames))

def progress_bar(current, total, bar_length=20):
    fraction = current / total

    arrow = int(fraction * bar_length - 1) * '-' + '>'
    padding = int(bar_length - len(arrow)) * ' '

    ending = '\n' if current == total else '\r'

    print(f'Progress: [{arrow}{padding}] {int(fraction*100)}%', end=ending)

In [12]:
images_path = 'aligned_train_try2\\'
img_filenames = [images_path + f for f in os.listdir(images_path) if f.endswith('.jpg')]
for img_filename in img_filenames:
    # img = cv.imread(img_filename)

    # scale = 0.5 # percent of original size
    # width = int(img.shape[1] * scale)
    # height = int(img.shape[0] * scale)
    # dim = (width, height)
    
    # img = cv.resize(img, dim)
    # r = cv.selectROI(img)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    # cropped_image = img[int(r[1]) : int(r[1] + r[3]), 
    #                     int(r[0]) : int(r[0] + r[2])]
    # print(r)
    img = cv.imread(img_filename)

    cropped_image = img[273 * 2 : (273 + 960) * 2,
                        547 * 2 : (547 + 956) * 2]
    
    cv.imwrite('aligned_cropped\\' + img_filename.split('\\')[-1], cropped_image)